<center><img src='https://drive.google.com/uc?export=view&id=1qJ8NqAZolTBQY7lN-deZ8xEsU3dlUiLz' width=200></center>


<h6><center></center></h6>

<h1>
<hr style=" border:none; height:3px;">
<center>Recherche d'Information et traitement de données massives </center>
    <center> Lab 3 : Modèles de recherche Probabilistes </center>
<hr style=" border:none; height:3px;">
</h1>

L'objectif de cette séance est de mettre en oeuvre les différents modèles de recherche dits "probabilistes" vus en cours 3. La première partie du lab est la mise en oeuvre des différents modèles sur le corpus TIME. La dernière partie correspond à des exercices d'application de cours disponibles sur EDUNAO et à faire en dehors de la séance d'aujourd'hui. 

## Partie 1  : Mise en oeuvre sur la collection TIME

Dans cette partie, il s'agit de **mettre en oeuvre les différents modèles probabilistes sur la collection TIME** en utilisant : 
 + la représentation de la collection sous la forme d'un index inversé et 
 + des requêtes ayant subies les mêmes traitements que ceux appliqués aux documents lors de la phase d'indexation. 

En effet, il est nécessaire de garantir que l'espace de représentation de la requête et du document sont les mêmes. Les modèles probabilistes font partis de l'ensemble des modèles de recherche en RI. Durant le LAB2 vous avez mis en pratique les modèles dits booléens et vectoriels. Ces derniers modélisent la pertinence d'un document par rapport à une requête en considérant respectivement un degré de pertinence binaire ou de similarité (score basé sur le produit vectoriel entre la représentation du document et celle de la requête). 

Les modèles probabilistes quant à eux poussent un peu plus loin la finesse de la modélisation et le calcul de la pertinence. Le principe général des modèles probabilistes consiste à inférer, étant données d'une part la représentation d'un document et d'autre part la représentation d'une requête, la probabilité de pertinence du document sachant la requête. L'idée principale est que plus la probabilité de pertinence d'un document de la collection est élevée plus le document répond au besoin de l'utilisateur tel que formulé par sa requête. C'est ainsi que les probabilités sont utilisées pour l'ordonnancement des documents par ordre de pertinence. Nous allons mettre en oeuvre deux modèles probabilistes populaires : **MIB** et **BM25** en implémentant pour chacun leur fonction score d'ordonnancement, toutes deux vues en cours.


**Préliminaires** 
Tout d'abord, nous allons charger l'index inversé de la collection TIME construit et sauvegardé sous la forme d'un fichier dans le LAB1. Les différents index sont disponibles [ici](./Index). Nous avons aussi mis dans le répertoire [Utils](./Utils) un ensemble de modules python permettant de charger les index. Ensuite, nous avons besoin d'avoir accès aux requêtes pré-processées comme cela a été fait durant le LAB2 par la construction de la fonction `pre_processed_query(query)`. Nous vous proposons de la ré-utiliser ainsi que les fonctions écrites dans le LAB1. Pour des raisons pratiques, nous vous fournissons ces fonctions dans le fichier [Lab12.py](./Utils/Lab12.py) du répertoire [Utils](./Utils). Pour les importer, si vous réalisez votre lab sur colab, il va falloir suivre les instructions de la section qui va suivre. Cependant, **si vous effectuez votre lab en local, vous pouvez sauter cette section**. 


**Instructions pour permettre l'importation des fonctions fournies et du répertoire Utils pour le cas du lab effectué sur colab :**



1.   Dans le dossier Lab3_ModelesDeRecherche téléchargé depuis git, vous trouverez une archive Archive_Lab3.zip.
2.   Ouvrir le panneau Fichiers de colab (c'est à dire cliquez sur le logo en forme de dossier à gauche) et cliquez sur le premier bouton en haut à gauche vous permettant de télécharger. Téléchargez ainsi l'Archive_Lab3.zip dans votre colab. Si besoin actualiser jusqu'à voir l'archive apparaître dans le panneau. 
3.   Exécutez les deux céllules de code suivantes pour déziper l'archive et installer à nouveau la librairie nltk. 

In [ ]:
!unzip Archive_Lab3.zip

In [ ]:
!pip install nltk
import nltk
nltk.download()

La partie d'importation suplémentaire pour les personnes réalisant leur lab sur colab est ici terminée. La suite est commune à tous.

In [ ]:
from Utils.saveandload_pickle import *
from Utils.Lab12 import *

#chargement des index inverses (document et frequence)
document_index = load_inverted_index_pickle('./Index/index_document.pickle')
frequence_index = load_inverted_index_pickle('./Index/index_frequence.pickle')
#print(document_index)

#chargement de la collection 
collection_TIME = loadData("./Data/Time/TIME.ALL")
pre_processed_collection_TIME = collection_lemmatize(remove_stop_words(tokenize_Regexp_corpus(collection_TIME),"./Data/Time/TIME.STP"))

#Exemple de requete pre-processee
pre_processed_query("KENNEDY ADMINISTRATION PRESSURE ON NGO DINH DIEM TO STOP")

## Modèle d'Indépendance Binaire (MIB)

**Fonction d'ordonnancement probabiliste du MIB ($RSV^{MIB}$)**

La formulation de la **fonction d'ordonnancement du Modèle d'Indépendance Binaire** est :

$$Score\_MIB(d,q)=\sum_{j:t_{j}=r_{j}=1} log \left( \frac{p_{j} (1-s_{j})}{s_{j} (1-p_{j})} \right)$$

où,
 + $r_j=1 \,(\mbox{resp}. t_j=1)\, \mbox{signifie la présence du}\; j \, \mbox{ème terme}\, t_j\, \mbox{du vocabulaire}\, (j \in \{1, \ldots , V \})\, \mbox{dans la requête}\, q\, (\mbox{resp. dans le document}\, d)$ ;
 + $p_j = P(t_{j}=1|R)\, \mbox{la probabilité de pertinence du terme}\, t_j$ ;
 + $s_j = P(t_{j}=1|NR)\, \mbox{la probabilité de non-pertinence du terme}\, t_j$.


Nous avons vu en cours comment théoriquement calculer les probabilités, $p_j\, \mbox{et} \, s_j$ à l'aide d'une table de contingence des occurrences des documents dans la collection pour une requête donnée. Cependant pour pouvoir compléter cette table nous avons besoin d'avoir accès à des données annotées (données d'apprentissage), c'est-à-dire pour lesquelles nous connaisons le résultat de pertinence et où nous avons accès à l'information de pertinence. Ici ce n'est pas  notre cas. Nous allons donc devoir envisager une alternative. Une solution proposée dans le cours est d'utiliser l'approximation suivante :
$$s_j \approx \frac{df_{t_j}}{N}$$
où, $N$ est le nombre de documents de la collection et $df_{t_j}$ le nombre de documents de la collection contenant le terme $t_j$.
Quant à $p_j$ plusieurs possibilités d'approximation existent : 
 + considérer $p_j$ constante, par exemple : 0.5 si aucune information n'est disponnible [Croft and Harper, 79] ;
 + considérer $p_j$ proportionnelle à la probabilité d'occurrence dans la collection ;
 + considérer $p_j$ proportionnelle au logarithme de la probabilité d'occurence dans la collection [GREIFF, Sigir 98].

1- Ecrire une fonction `get_sj(term,index_frequence,nb_doc)` qui calcule l'approximation de $s_j$ pour un terme $t_j$. En entrée de la fonction vous pouvez utiliser le terme `term` et l'index de fréquence `index_frequence` précédemment chargé ainsi que le nombre de documents de la collection `nb_doc`. Il est égal à 423 pour la collection TIME mais si vous le souhaitez, vous pouvez le récupérer via le dictionnaire de statistiques construit dans le LAB2.


In [ ]:
from math import *

def get_sj(term,index_frequence,nb_doc):
    #A completer
               
get_sj("NASSAU",frequence_index,423)

#print(frequence_index["NASSAU"].keys())

2- Ecrire une fonction `score_MIB_1 (doc_ID, query, index_frequence, nb_doc)` qui calcule le score de pertinence du modèle MIB pour un document identifié dans l'`index_frequence` par son `doc_ID` et une requête `query`, tous donnés en entrée pour le cas où $p_j=0.5$. 

In [ ]:
def score_MIB_1 (doc_ID, query, index_frequence, nb_doc):
    pj=0.5
    cj_all= [] #liste vide de stockage des cj
    query_pre_processed = pre_processed_query(query)
    #A completer
    return sum(cj_all) #retourn la somme de la liste

    

3- Appliquer cette fonction pour le cas de la requête "KENNEDY ADMINISTRATION PRESSURE ON NGO DINH DIEM TO STOP" et un document au choix de la collection. N'hésitez pas à changer de document 2 ou 3 fois et à répéter le test. 

In [ ]:
score_MIB_test=score_MIB_1(#A completer)

print(score_MIB_test)


4- **(question optionnelle)** Ecrire les fonctions `score_MIB_2 ()` et `score_MIB_3 ()` correspondant aux deux autres solutions de calcul de $p_j$.

5- Ecrire une fonction `ranking_score_MIB_1 (query, index_frequence, nb_doc)` qui calcule pour une requête donnée en entrée le `score_MIB_1` avec tous les documents "pertinents" de la collection et renvoie en sortie la liste des couples : document pertinent puis score, ordonnée par score de pertinence décroissant. 

N.B : un document est dit ici "pertinent" dans le sens où il contient au moins un terme de la requête. Cela n'a rien à voir avec la notion de pertinence du cours.

In [ ]:

def ranking_score_MIB_1 (query, index_frequence, nb_doc):
    query_pre_processed = pre_processed_query(query)
    score = {}
    #A completer
    #Ordonner
    ordered_scores = #A completer
    return ordered_scores

#Application :
ranking_score_MIB_1("KENNEDY ADMINISTRATION PRESSURE ON NGO DINH DIEM TO STOP",frequence_index,423)

6- **(question optionnelle)** Faire de même pour les cas des fonctions `score_MIB_2 ()` et `score_MIB_3 ()`.

7- **(question optionnelle)** Ecrire une fonction globale pour le "ranking" de documents pertinents par score MIB avec un paramètre d'entrée permettant de choisir parmi les fonctions de score `score_MIB_1 ()`, `score_MIB_2 ()` et `score_MIB_3 ()`.

8- Quelle information présente dans l'index inversé n'est pas utilisée par la modélisation MIB ? Quelle modélisation permet d'utiliser cette information pertinente ?

**Réponse**: #A completer

## Modèle BM 25 ("Best Match")



Le modèle BM 25, aussi nommé OKAPI BM 25, est une référence dans le développement des systèmes de recherche. Il est fondé sur un "principe d'indexation probabiliste" dont l'idée sous-jacente est qu'un bon descripteur de document est un terme assez fréquent dans ce document mais qui est relativement rare dans la collection. Cette idée a pour notion de base la division des documents en deux ensembles élite $E$ et non élite $\bar{E}$ et aussi les notions de probabilités de pertinence d'un terme $p_j$ et $s_j$. En effet, sous le modèle BM 25 nous allons réécrire la fonction de score du modèle MIB en y intégrant :
   + la notion d'élitisme des termes ;
   + l'hypothèse que la fréquence d'un mot dans un document ne dépend que de l'appartenance du document à l'ensemble élite.

En particulier, cela signifie que nous allons considérer que,
   $p_j = P(TF_{j}=tf_{t_j,d} |R )$
et que, 
   $s_j = P(TF_{j}=tf_{t_j,d} |NR )$

avec $tf_{t_j,d}$ le nombre d'occurrences du terme dans le document. Quand le terme est absent, $tf_{t_j,d}=0$. Ainsi, quand le terme est présent, le modèle tiendra aussi compte de sa fréquence d'apparition contrairement au modèle de base MIB.

**Fonction d'ordonnancement probabiliste du BM 25**

La formulation, ou plus exactement son approximation, de la **fonction d'ordonnancement du Modèle BM 25** est :

$$Score\_BM25(d,q) = \sum_{j:t_j=r_j=1} \frac{(k_1 + 1) \times tf_{t_j,d}}{k_1 ((1-b) + b \frac{L_d}{m} )+ tf_{t_j,d}} \times \frac{(k_3 + 1) \times tf_{t_j,q}}{k_3 + tf_{t_j,q}} \times \log \frac{N - df_{t_j} + 0.5}{df_{t_j} + 0.5}$$

où,
 + $r_j=1$ (resp. $t_j=1$) signifie la présence du $j$ème terme $t_j$ du vocabulaire ($j \in \{1, \ldots , V \}$) dans la requête $q$ (resp. dans le document $d$) ;
 + $p_j = P(TF_{j}=tf_{t_j,d} |R )$ la probabilité de pertinence du terme $t_j$ ;
 + $s_j = P(TF_{j}=tf_{t_j,d} |NR )$ la probabilité de non-pertinence du terme $t_j$ ;
 + $df_{t_j}$ est le nombre de documents de la collection contenant le terme $t_j$ ;
 + $L_d$ est la longueur du document $d$ ;
 + $ m = \frac{1}{N} \sum_{d \in \mathcal{C}} L_d$ est la moyenne des tailles des documents de la collection ;
 + $k_1$ est le paramètre contrôlant la prise en compte de la fréquence des termes, par défaut $k_1 = 1.2$ ;
 + $b$ est le paramètre contrôlant la prise en compte de la longueur, par défaut $b = 0.75$ ;
 + $k_3$ est le paramètre contrôlant la prise en compte de la fréquence $tf_{t_i,q}$, par défaut on fixe $k_3 = 1000$ ;
 + $tf_{t_j,q}$ est le nombre d'occurrences du terme $t_j$ dans la requête $q$.


A ce stade du LAB, nous savons calculer $tf_{t_j,d}$, $tf_{t_j,q}$, $df_{t_j}$ et nous connaissons $N$. Pour les éléments $k_1$, $k_2$, $k_3$ et $b$ il nous est possible de les fixer aux valeurs par défaut. Il nous reste donc à construire une fonction calculant $tf_{t_j,q}$ puis à calculer $L_d$ et $m$. Nous aurons alors à disposition tout le nécessaire pour calculer la fonction score du modèle BM 25. 

9- Ecrire une fonction `get_tf_q (term, query)` permettant de calculer le nombre d'occurrence  d'un terme `term` dans une requête `query` tous deux donnés en entrée.

In [ ]:
def get_tf_q (term, query):
    #A completer  
    return #Acompleter

#Application :
test = get_tf_q("NGO", "KENNEDY ADMINISTRATION PRESSURE ON NGO DINH DIEM TO STOP")
print(test)


10- Ecrire une fonction `create_dict_size_docs_collection(pre_processed_collection_TIME)` qui construit à partir de la collection pré-processée (chargée au début de ce Lab) un dictionnaire où les clés sont les identifiants des documents et les valeurs les longueurs $L_d$ des documents. Puis calculer la longueur moyenne `m` des tailles des documents de la collection.

In [ ]:
import numpy as np

def create_dict_size_docs_collection(pre_processed_collection_TIME):
    size_docs = {}
    #A completer
    return size_docs

#Application :
#Creation du dictionnaire tel que pour chaque doc_ID on a sa longueur L_d
dict_size_docs = #A completer

#print(dict_size_docs)

#Calcul de la moyenne des longueurs de documents de la collection
sizes_docs = #A completer
m = np.mean(sizes_docs)
print(m)

11- Ecrire une fonction `score_BM25 (doc_ID, query, index_frequence, nb_doc, dict_size_docs, m)` qui calcule le score de pertinence du modèle BM25 pour un document identifié dans l'`index_frequence` par son `doc_ID` et une requête `query`. En entrée, vous pouvez utiliser les éléments créés à la question précédentes : `m`et `dict_size_docs`.

In [ ]:


def score_BM25 (doc_ID, query, index_frequence, nb_doc, dict_size_docs, m):
    #valeurs par defaut
    k1 = #A completer
    k3 = #A completer
    b = #A completer
    query_pre_processed = #A completer
    score_j_all = []
    #A completer
    return sum(score_j_all)

#Application :
score_BM25_test= score_BM25(#A completer)

print(score_BM25_test)

12- Ecrire une fonction `ranking_score_BM25 (query, index_frequence, nb_doc)` qui calcule pour une requête donnée en entrée le `score_BM25` avec tous les documents "pertinents" de la collection et renvoie en sortie la liste des couples : document pertinent puis score, ordonnée par score de pertinence décroissant. 

N.B : un document est dit ici "pertinent" dans le sens où il contient au moins un terme de la requête. Cela n'a rien à voir avec la notion de pertinence du cours.


In [ ]:
def ranking_score_BM25 (query, index_frequence, nb_doc):
    #A completer
    
    #Ordonner
    ordered_scores = #A completer
    return ordered_scores

#Application :
ranking_score_BM25(#A completer)

13- Comparer les deux ranking renvoyés par chacun des modèles de recherche probabilistes pour une même requête `"KENNEDY ADMINISTRATION PRESSURE ON NGO DINH DIEM TO STOP"`.

14- Comparer les rankings renvoyés par tous les modèles de recherche vus durant les Labs (probabilistes et vectoriel) pour la même requête `"KENNEDY ADMINISTRATION PRESSURE ON NGO DINH DIEM TO STOP"`. Que pouvez-vous dire ? Comment pourrait-on evaluer et comparer ces différents modèles ?


## Partie 2 : Exercices (Série sur EDUNAO - A faire en dehors de la séance de TD)